In [1]:
import importlib
from main import TimeLoopExperimentController
import architectures.architecture_strategy
import architectures.architecture_constants
import architecture_results.derived_metrics_evaluator
import matplot_results_plotter.matplot_results_plotter

from architectures.architecture_constants import Architecture, GPUMemoryScale, RackSize, PEsConfig, base_config, resnet_18_layers

import matplotlib.pyplot as plt

importlib.reload(architectures.architecture_strategy)
importlib.reload(architectures.architecture_constants)
importlib.reload(architecture_results.derived_metrics_evaluator)
importlib.reload(matplot_results_plotter.matplot_results_plotter)

<module 'matplot_results_plotter.matplot_results_plotter' from '/home/workspace/matplot_results_plotter/matplot_results_plotter.py'>

In [2]:
# debug currently set to True, set to False to run
# controller = TimeLoopExperimentController(True)
controller = TimeLoopExperimentController(False)
controller.run_all("persisted_results/results_May 03, 2025 04:16:57 PM EST")
# controller.plot_results()

Starting Timeloop with configuration:
  - arch: Architecture.Tensor_Parallel
  - scale: GPUMemoryScale.MEMORY_4MB
  - rack: RackSize.RACK_4
  - peConfig: PEsConfig.PE_1
Running layer 1: l1_conv1 which occurs 1 times
[INFO] 2025-05-04 18:15:33,914 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Error running layer l1_conv1: 

Timeloop mapper failed with return code 0. Please check the output files in ./output_dir for more information. To debug, you can edit the file:
	./output_dir/parsed-processed-input.yaml
and run 
	tl mapper ./output_dir/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.
Running layer 1: l2_conv2_stride2 which occurs 1 times
Running layer 1: l3-l7_conv2_stride1 which occurs 5 times
Running layer 1: l8_conv3_stride2 which occurs 1 times
Running layer 1: l9-l15_conv3_stride1 which occurs 7 times
Running layer 1: l16_conv4_stride2 which occurs 1 times
Running layer 1: l17-l27_conv4_stride1 which occurs 11 times
Running layer 1: l28_conv5_stride2 which occurs 1 times
Running layer 1: l29-l33_conv5_stride1 which occurs 5 times
{'energy': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Error running layer l1_conv1: 

Timeloop mapper failed with return code 0. Please check the output files in ./output_dir for more information. To debug, you can edit the file:
	./output_dir/parsed-processed-input.yaml
and run 
	tl mapper ./output_dir/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.
Running layer 1: l2_conv2_stride2 which occurs 1 times
Running layer 1: l3-l7_conv2_stride1 which occurs 5 times
Running layer 1: l8_conv3_stride2 which occurs 1 times
Running layer 1: l9-l15_conv3_stride1 which occurs 7 times
Running layer 1: l16_conv4_stride2 which occurs 1 times
Running layer 1: l17-l27_conv4_stride1 which occurs 11 times
Running layer 1: l28_conv5_stride2 which occurs 1 times
Running layer 1: l29-l33_conv5_stride1 which occurs 5 times
{'energy': [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

...

Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 1
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 2
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 3
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 4
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 5
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 6
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 7
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 8
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 9
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 10
Adding energy and cycle stats for l17-l27_conv4_stride1 occurence 11
Running layer 28: l28_conv5_stride2 which occurs 1 times
[INFO] 2025-05-04 22:20:51,359 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Adding energy and cycle stats for l28_conv5_stride2 occurence 1
Running layer 29: l29-l33_conv5_stride1 which occurs 5 times
[INFO] 2025-05-04 22:23:04,362 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Adding energy and cycle stats for l29-l33_conv5_stride1 occurence 1
Adding energy and cycle stats for l29-l33_conv5_stride1 occurence 2
Adding energy and cycle stats for l29-l33_conv5_stride1 occurence 3
Adding energy and cycle stats for l29-l33_conv5_stride1 occurence 4
Adding energy and cycle stats for l29-l33_conv5_stride1 occurence 5
{'energy': [5398813.27, 4387439.74, 4387439.74, 4387439.74, 4387439.74, 4387439.74, 4387439.74, 2233710.84, 4351952.19, 4351952.19, 4351952.19, 4351952.19, 4351952.19, 4351952.19, 4351952.19, 2196255.21, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 4335006.34, 2178853.72, 4329412.85, 4329412.85, 4329412.85, 4329412.85, 4329412.85], 'cycles': [78675968, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 28901376, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 28901376, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 57802752, 

In [ ]:
# debug currently set to True, set to False to run
# controller = TimeLoopExperimentController(True)
controller = TimeLoopExperimentController(False)
controller.run_single(Architecture.Data_Parallel, GPUMemoryScale.MEMORY_16MB, RackSize.RACK_4, PEsConfig.PE_1, "persisted_results/results_May 03, 2025 04:16:57 PM EST")
# controller.run_single(Architecture.Base, GPUMemoryScale.MEMORY_16MB, RackSize.RACK_4, PEsConfig.PE_4, "persisted_results/milestone3")
# controller.run_single(Architecture.Tensor_Parallel, GPUMemoryScale.MEMORY_16MB, RackSize.RACK_4, PEsConfig.PE_4, "persisted_results/milestone3")
# # controller.plot_results()

Starting Timeloop with configuration:
  - arch: Architecture.Data_Parallel
  - scale: GPUMemoryScale.MEMORY_16MB
  - rack: RackSize.RACK_4
  - peConfig: PEsConfig.PE_1
Running layer 1: l1_conv1 which occurs 1 times
[INFO] 2025-05-04 09:03:39,594 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Adding energy and cycle stats for l1_conv1 occurence 1
Running layer 2: l2_conv2_stride2 which occurs 1 times
[INFO] 2025-05-04 09:05:45,578 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Adding energy and cycle stats for l2_conv2_stride2 occurence 1
Running layer 3: l3-l7_conv2_stride1 which occurs 5 times
[INFO] 2025-05-04 09:09:16,552 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Adding energy and cycle stats for l3-l7_conv2_stride1 occurence 1
Adding energy and cycle stats for l3-l7_conv2_stride1 occurence 2
Adding energy and cycle stats for l3-l7_conv2_stride1 occurence 3
Adding energy and cycle stats for l3-l7_conv2_stride1 occurence 4
Adding energy and cycle stats for l3-l7_conv2_stride1 occurence 5
Running layer 8: l8_conv3_stride2 which occurs 1 times
[INFO] 2025-05-04 09:12:53,190 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v
